In [1]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
%run 05_utils.ipynb
%run 01_Credentials.ipynb
import os 
import datetime

In [2]:
conn,cur = connect_to_db("GH_Sensors_DWH") 

Connecting to Billing_DWH..... 
Connection to PostgerSQL is done successfuly
conn and cur are created and successfuly returned


In [3]:
try:
    # read data from dwh tables convert it into pandas data frame  
    df_fa_measur = pd.read_sql("SELECT * FROM sensor_measurements", conn)
    df_dim_sensor = pd.read_sql("SELECT * FROM sensor_dimension", conn)
    df_dim_loc = pd.read_sql("SELECT * FROM location_dimension", conn)
    df_dim_date = pd.read_sql("SELECT * FROM time_dimension", conn)

    # save pandas data frames into csv files 
    df_fa_measur.to_csv("D:\ITI matrial\Week_12_DWH\GH_Real_time_Data_Streaming_Pipeline\CSV_data\measurements.csv", index=False)
    df_dim_sensor.to_csv("D:\ITI matrial\Week_12_DWH\GH_Real_time_Data_Streaming_Pipeline\CSV_data\dim_sensor.csv", index=False)
    df_dim_loc.to_csv("D:\ITI matrial\Week_12_DWH\GH_Real_time_Data_Streaming_Pipeline\CSV_data\dim_location.csv", index=False)
    df_dim_date.to_csv("D:\ITI matrial\Week_12_DWH\GH_Real_time_Data_Streaming_Pipeline\CSV_data\dim_date.csv", index=False)
except ValueError:
    print('there is an error occared')
    

In [11]:
# Truncate the existing data after exporting existent data from it to csv files to make it clear from the next day transaction
# cur.execute("""TRUNCATE TABLE sensor_measurements""")
# cur.execute("""TRUNCATE TABLE time_dimension""")
# cur.execute("""TRUNCATE TABLE location_dimension""")
# cur.execute("""TRUNCATE TABLE sensor_dimension""")

In [4]:
# Snowflake connection parameters
conn = snowflake.connector.connect(
    user='MEldeeb',
    password= os.environ.get('snowflake_pass'),
    account='VMFPMHB-OR45087', 
    warehouse='COMPUTE_WH',
    database='GH_DWH',
    schema='PUBLIC'
)
cur = conn.cursor()


In [7]:
# Create a stage in Snowflake to hold the CSV files read from the ODS

#load dim_location.csv into GH_DATA_STAGE data stage
cur.execute(r"""
    PUT 'file://D:/ITI matrial/Week_12_DWH/GH_Real_time_Data_Streaming_Pipeline/CSV_data/dim_location.csv' 
    @GH_DATA_STAGE 
    AUTO_COMPRESS=TRUE
""")


#load dim_date.csv into GH_DATA_STAGE data stage
cur.execute(r"""
    PUT 'file://D:/ITI matrial/Week_12_DWH/GH_Real_time_Data_Streaming_Pipeline/CSV_data/dim_date.csv' 
    @GH_DATA_STAGE 
    AUTO_COMPRESS=TRUE
""")

#load dim_sensor.csv into GH_DATA_STAGE data stage
cur.execute(r"""
    PUT 'file://D:/ITI matrial/Week_12_DWH/GH_Real_time_Data_Streaming_Pipeline/CSV_data/dim_sensor.csv' 
    @GH_DATA_STAGE 
    AUTO_COMPRESS=TRUE
""")

#load fact_measurements.csv into GH_DATA_STAGE data stage
cur.execute(r"""
    PUT 'file://D:/ITI matrial/Week_12_DWH/GH_Real_time_Data_Streaming_Pipeline/CSV_data/measurements.csv' 
    @GH_DATA_STAGE 
    AUTO_COMPRESS=TRUE
""")



In [9]:
# Truncate the existing tables before loading new data 
# we use this in the first time only
cur.execute("""TRUNCATE TABLE FACT_MEASUREMENTS""")
cur.execute("""TRUNCATE TABLE DIM_DATE""")
cur.execute("""TRUNCATE TABLE DIM_LOCATION""")
cur.execute("""TRUNCATE TABLE DIM_SENSOR""")

In [10]:
# Copy data from the stage to the Snowflake tables 

# Copy data into DIM_LOCATION table from the staged file (dim_date.csv.gz)
cur.execute("""
    COPY INTO DIM_DATE FROM @GH_DATA_STAGE/dim_date.csv.gz
    FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1);
""")

# Copy data into DIM_SENSOR table from the staged file (dim_sensor.csv.gz)
cur.execute("""
    COPY INTO DIM_LOCATION FROM @GH_DATA_STAGE/dim_location.csv.gz
    FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1);
""")


# Copy data into DIM_LOCATION table from the staged file (dim_location.csv.gz)
cur.execute("""
    COPY INTO DIM_SENSOR FROM @GH_DATA_STAGE/dim_sensor.csv.gz
    FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1);
""")

# Copy data into FACT_MEASUREMENTS table from the staged file (measurements.csv.gz)
cur.execute("""
    COPY INTO FACT_MEASUREMENTS FROM @GH_DATA_STAGE/measurements.csv.gz
    FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1);
""")


In [11]:
conn.close()